# An offline demo for the "Learning-based Video Motion Magnification" (ECCV 2018)

# Official repo: https://github.com/12dmodel/deep_motion_mag
# My repo: https://github.com/ZhengPeng7/motion_magnification_learning-based

## Make sure to set your python Kernel for notebook. 

## Preparations:


### Install python packages

In [1]:
!pip install --quiet -r requirements.txt
!pip install --quiet gdown mediapy

### Download and load the well-trained weights:

In [2]:
!wget https://github.com/ZhengPeng7/motion_magnification_learning-based/releases/download/v1.0/magnet_epoch12_loss7.28e-02.pth

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [4]:
from magnet import MagNet
from callbacks import gen_state_dict
from config import Config


# config
config = Config()
# Load weights
weights_path = 'magnet_epoch12_loss7.28e-02.pth'
ep = int(weights_path.split('epoch')[-1].split('_')[0])
state_dict = gen_state_dict(weights_path)

model_test = MagNet().cuda()
model_test.load_state_dict(state_dict)
model_test.eval()
print("Loading weights:", weights_path)

Please load train_mf.txt if you want to do training.


FileNotFoundError: [Errno 2] No such file or directory: 'magnet_epoch12_loss7.28e-02.pth'

# Preprocess

Make the video to frameAs/frameBs/frameCs.

Let's take the guitar.mp4 as an example.

In [ ]:
# Download some example video from my google-drive or upload your own video.
# !gdown 1hNZ02vnSO04FYS9jkx2OjProYHIvwdkB  # guitar.avi
!gdown 1XGC2y4Lshd9aBiBxwkTuT_IA79n-WNST  # baby.avi
# !gdown 1QGOWuR0swF7_eHharTztlkEDz0hlfmU4  # zhiyin.mp4

# Set VIDEO_NAME here, e.g., guitar, baby

In [ ]:
# Turn the video into frames and make them into frame_ACB format.
file_to_be_maged = 'baby.avi'   # 'guitar.avi'
video_name = file_to_be_maged.split('.')[0]
video_format = '.' + file_to_be_maged.split('.')[-1]


sh_file = 'VIDEO_NAME={}\nVIDEO_FORMAT={}'.format(video_name, video_format) + """


mkdir ${VIDEO_NAME}
ffmpeg -i ${VIDEO_NAME}${VIDEO_FORMAT} -f image2 ${VIDEO_NAME}/%06d.png
python make_frameACB.py ${VIDEO_NAME}
mkdir test_dir
mv ${VIDEO_NAME} test_dir
"""
with open('test_preproc.sh', 'w') as file:
  file.write(sh_file)

!bash test_preproc.sh

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1597178665428/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libsw

## Test


In [ ]:
import os
import sys
import cv2
import torch
import numpy as np
from data import get_gen_ABC, unit_postprocessing, numpy2cuda, resize2d


# testsets = '+'.join(['baby', 'guitar', 'gun', 'drone', 'cattoy', 'water'][:])
for testset in [video_name]:
    dir_results = 'res_' + testset
    if not os.path.exists(dir_results):
        os.makedirs(dir_results)

    config.data_dir = 'test_dir'
    data_loader = get_gen_ABC(config, mode='test_on_'+testset)
    print('Number of test image couples:', data_loader.data_len)
    vid_size = cv2.imread(data_loader.paths[0]).shape[:2][::-1]

    # Test
    #Magification factor here:{amp}
    for amp in [10, 25, 50]:
        frames = []
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        for idx_load in range(0, data_loader.data_len, data_loader.batch_size):
            if (idx_load+1) % 100 == 0:
                print('{}'.format(idx_load+1), end=', ')
            batch_A, batch_B = data_loader.gen_test()
            amp_factor = numpy2cuda(amp)
            for _ in range(len(batch_A.shape) - len(amp_factor.shape)):
                amp_factor = amp_factor.unsqueeze(-1)
            with torch.no_grad():
                y_hats = model_test(batch_A, batch_B, 0, 0, amp_factor, mode='evaluate')
            for y_hat in y_hats:
                y_hat = unit_postprocessing(y_hat, vid_size=vid_size)
                frames.append(y_hat)
                if len(frames) >= data_loader.data_len:
                    break
            if len(frames) >= data_loader.data_len:
                break
        data_loader = get_gen_ABC(config, mode='test_on_'+testset)
        frames = [unit_postprocessing(data_loader.gen_test()[0], vid_size=vid_size)] + frames

        # Make videos of framesMag
        video_dir = os.path.join(dir_results, testset)
        if not os.path.exists(video_dir):
            os.makedirs(video_dir)
        FPS = 30
        video_save_path = os.path.join(video_dir, '{}_amp{}{}'.format(testset, amp, video_format))
        out = cv2.VideoWriter(
            video_save_path,
            cv2.VideoWriter_fourcc(*'DIVX'),
            FPS, frames[0].shape[-2::-1]
        )
        for frame in frames:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            cv2.putText(frame, 'amp_factor={}'.format(amp), (7, 37),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
            out.write(frame)
        out.release()
        print('{} has been done.'.format(video_save_path))

Number of test image couples: 300
100, 200, 300, res_baby/baby/baby_amp10.avi has been done.
100, 200, 300, res_baby/baby/baby_amp25.avi has been done.
100, 200, 300, res_baby/baby/baby_amp50.avi has been done.


In [ ]:
# Play the amplified video here
from glob import glob
import mediapy


video_save_paths = [file_to_be_maged] + sorted(glob(os.path.join(dir_results, testset, '*')), key=lambda x: int(x.split('amp')[-1].split('.')[0]))

video_dict = {}
for video_save_path in video_save_paths[:]:
  video_dict[video_save_path.split('/')[-1]] = mediapy.read_video(video_save_path)
mediapy.show_videos(video_dict, fps=FPS, width=250, codec='gif')